# Notebook use to test backend and tensorflow fonctionnalities

In [1]:
48600/25

1944.0

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.models import Sequential
from keras.layers import Dense, LSTM, RNN, SimpleRNN, GRU, Activation, Flatten, Conv1D, TimeDistributed
from keras import optimizers
from keras.layers import Bidirectional
from keras import backend as K

from keras.models import Sequential
import keras.backend as K
#import tensorflow as tf
%matplotlib inline
n = 100         # sample size
x = np.random.random((65,16))*50    #input
y = (4*(x-0.5))[:,0]       #output
dy = 8*(x-0.5)       #derivative of output wrt the input

Using TensorFlow backend.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import keras
import datetime

#from tqdm import tqdm
from math import ceil
from generator import Diff_Generator
from preprocess import ProdKernel, FKernel, DictPrepross
from utils import Load_FLX_dict, Plot_Batch

data_folder="Data"

from keras.models import Sequential
from keras.layers import Dense, LSTM, RNN, SimpleRNN, GRU, Activation, Flatten, Conv1D, TimeDistributed
from keras import optimizers
from keras.layers import Bidirectional
from keras import backend as K

##### Dictionnary
D = Load_FLX_dict()
##### Kernels
Klist = []
##### b_size
batch_size= 64

train_generator = Diff_Generator( folder=data_folder, batch_size=batch_size, train=True, \
                                 preprocess_x=[D]+Klist)
validation_generator = Diff_Generator( folder=data_folder, batch_size=batch_size, \
                                train=False, preprocess_x=[D]+Klist, custom_b_p_e = 50)

n_channel = len(train_generator.variables)
modelbd = Sequential()
#modelbd.add(Input(batch_shape=(64,72, n_channel)))
modelbd.add(Bidirectional(LSTM(128, return_sequences=True, use_bias=False),input_shape=(72, n_channel)))
modelbd.add(Conv1D(50, use_bias=False,kernel_size=10 ,padding='same'))
modelbd.add(Activation('relu'))
modelbd.add(Conv1D(20, kernel_size=3 ,padding='same'))
modelbd.add(Activation('relu'))
modelbd.add(Dense(1))
modelbd.add(Flatten())

rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=1.e-5)
modelbd.compile(loss='mse', optimizer=rmsprop)
modelbd.summary()
model_folder = "TrainedModels"
prefix = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
archi = "Bidir"
x_test_1 = train_generator[0][0]

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_1 (Bidirection (None, 72, 256)           142336    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 72, 50)            128000    
_________________________________________________________________
activation_1 (Activation)    (None, 72, 50)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 72, 20)            3020      
_________________________________________________________________
activation_2 (Activation)    (None, 72, 20)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 72, 1)             21        
_________________________________________________________________
flat

In [20]:
k_constants = K.constant(x_test_1[:,:,:-1])
input11 = keras.layers.Input(batch_shape=(64,1))
input12 = keras.layers.RepeatVector(n=72)(input11)
out11 = keras.layers.Lambda(lambda x: K.concatenate([x,k_constants],axis=-1))(input12)
out12 = modelbd(out11)
out13 = keras.layers.Lambda(lambda x: x[:,0])(out12)
model1 = keras.models.Model(inputs=input11, outputs=out13)
print(model1.summary())
#k_constants = K.constant(x_test_1[:,:,:-1])
input21 = keras.layers.Input(batch_shape=(64,72, 1))
#input12 = keras.layers.RepeatVector(n=72)(input11)
out21 = keras.layers.Lambda(lambda x: K.concatenate([x,k_constants],axis=-1))(input21)
out22 = modelbd(out21)
model2 = keras.models.Model(inputs=input21, outputs=out22)
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (64, 1)                   0         
_________________________________________________________________
repeat_vector_6 (RepeatVecto (64, 72, 1)               0         
_________________________________________________________________
lambda_10 (Lambda)           (64, 72, 11)              0         
_________________________________________________________________
sequential_1 (Sequential)    multiple                  273377    
_________________________________________________________________
lambda_11 (Lambda)           (64,)                     0         
Total params: 273,377
Trainable params: 273,377
Non-trainable params: 0
_________________________________________________________________
None
_________________________________________________________________
Layer (type)                 Output Shape              Param #   

<tf.Tensor 'lambda_11/strided_slice:0' shape=(64,) dtype=float32>

In [23]:
import time
#gradients = K.gradients(modelbd.get_output_at(), modelbd.input)  
s = np.zeros((64,72,1))
x_test_1 = train_generator[0][0]
ts = x_test_1[:,0,-1].reshape(x_test_1.shape[0], 1)

t0=time.time()
gradients=[]
for i in range(72):    
    t=time.time()
    #######
    input11 = keras.layers.Input(batch_shape=(64,1))
    input12 = keras.layers.RepeatVector(n=72)(input11)
    out11 = keras.layers.Lambda(lambda x: K.concatenate([x,k_constants],axis=-1))(input12)
    out12 = modelbd(out11)
    out13 = keras.layers.Lambda(lambda x: x[:,i])(out12)
    model1 = keras.models.Model(inputs=input11, outputs=out12)
    ########
    gradients = K.gradients(model1.get_output_at(0), model1.input)
    grad0 = K.function( [model1.input] , [gradients[0]] )
    grad0([ts])
    print(time.time()-t)

#gradients = np.array(gradients)
#evaluated_gradients_1 = sess.run(gradients, feed_dict={model1.input: ts})
print(time.time() - t0)

5.244035959243774
5.12951922416687
5.343057632446289
5.306421279907227
5.51494026184082
5.511064052581787
5.687743425369263
5.7447509765625
6.55673360824585
5.845239162445068
6.007688999176025
6.042474985122681
6.2214155197143555
6.24210524559021
6.409037351608276
6.462530612945557
6.589289426803589
6.9732584953308105
7.055161714553833
7.132930040359497
7.277195930480957
7.367127895355225
8.383916854858398
7.576846361160278
7.825615167617798
7.8559465408325195
7.988591909408569
8.052626132965088
8.223589420318604
8.283080339431763
8.438371419906616
8.504306316375732
8.739555835723877
8.710597038269043
8.81786036491394
8.914911031723022
9.024802446365356
9.197700262069702
9.52592158317566
9.510283470153809
10.652295351028442
9.738377332687378
9.816979169845581
9.92486047744751
10.071983814239502
10.16982650756836
10.273815393447876
10.359548091888428
10.475645542144775
10.24144983291626
10.362910032272339
10.547290802001953
10.643597841262817
10.720527410507202
10.962612628936768
11.061

In [12]:
grad0 = K.function( [model1.input] , [gradients[i][0] for i in range(36)])
ts = x_test_1[:,0,-1].reshape(x_test_1.shape[0],1 )
t=time.time()

print(time.time()-t)

21.725669622421265


In [ ]:
grad0

In [30]:
evaluated_gradients_1[0][0]-a

array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]], dtype=float32)

In [11]:
#import tensorflow as tf
grad = tf.gradients(model1.get_output_at(0)[:,i], model1.input)

grad[0]
sess.run(grad[0], model1.input = ts )

<tf.Tensor 'gradients_148/repeat_vector_1/ExpandDims_grad/Reshape:0' shape=(64, 1) dtype=float32>

In [35]:
import tensorflow as tf
import time
#gradients = K.gradients(modelbd.get_output_at(), modelbd.input)  

s = np.zeros((64,72,1))
x_test_1 = train_generator[0][0]
t0=time.time()
sess = K.get_session()
for i in range(72):    
    t=time.time()

    gradients = K.gradients(model1.get_output_at(0)[:,i], model1.input)
    ts = x_test_1[:,0,-1].reshape(x_test_1.shape[0],1 )
    evaluated_gradients_1 = sess.run(gradients[0], feed_dict={model1.input: ts})
    print(i, time.time()-t)
    s[:,i] += evaluated_gradients_1;

tf.while_loop(
    cond,
    body,
    loop_vars,
    shape_invariants=None,
    parallel_iterations=10,
    back_prop=True,
    swap_memory=False,
    name=None,
    maximum_iterations=None,
    return_same_structure=False
)

    n = tf.shape(x)[0]
    loop_vars = [
        tf.constant(0, n.dtype),
        tf.TensorArray(dtype, size=n),
    ]
    
    parallel_iterations=10
    j=tf.constant(0)
    _, fx = tf.while_loop(
        lambda j, _: j < n,
        
        lambda j, result: (j + 1, result.write(j, f(x[j]))), loop_vars,
                    g = tf.gradients(model1.get_output_at(0)[:,i], model1.input)
                    g
        parallel_iterations=parallel_iterations
    )
    return fx.stack()

CancelledError: Session has been closed.

In [43]:
#gradients = K.gradients(modelbd.get_output_at(), modelbd.input)  
s = np.zeros((64,72,1))
x_test_1 = train_generator[0][0]

for i in range(10):
    print(i)
    
    gradients = K.gradients(modelbd.get_output_at(0)[:,i], modelbd.get_input_at(0)[:,:,10])
#    gradients = K.gradients(modelbd.get_output_at(0)[:,i], modelbd.get_input_at(0))
  
    #value of gradient for the first x_test
    #x_test_1 = x_test_1.reshape(1,72,11)
    sess = K.get_session()
    evaluated_gradients_1 = sess.run(gradients[0], feed_dict={modelbd.input: x_test_1})
    #print(evaluated_gradients_1)
    s+=evaluated_gradients_1;

gradients = K.gradients(modelbd.get_output_at(0), modelbd.input)
sess = K.get_session()
evaluated_gradients_1 = sess.run(gradients[0], feed_dict={modelbd.input: x_test_1})

0


AttributeError: Layer sequential_1 has multiple inbound nodes, hence the notion of "layer input" is ill-defined. Use `get_input_at(node_index)` instead.

In [6]:
# with all 72
print('Phase 2')
s2 = np.zeros((64,72, 1))
for i in range(10):
    gradients = K.gradients(model2.get_output_at(0)[:,i], model2.input)
    sess = K.get_session()
    ts = x_test_1[:,:,-1].reshape(x_test_1.shape[0], x_test_1.shape[1], 1 )
    t=time.time()
    evaluated_gradients_1 = sess.run(gradients[0], feed_dict={model2.input: ts})
    print(i, time.time()-t)
    s2+=evaluated_gradients_1;

NameError: name 'tf' is not defined

In [5]:
x_test_1 = train_generator[0][0]
test_generator = Diff_Generator(folder=data_folder, batch_size=16, \
                                train=False, shuffle=True, preprocess_x=[D]+Klist, custom_b_p_e = 50)
x,y = test_generator[0]

In [16]:
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (64, 72, 1)               0         
_________________________________________________________________
lambda_7 (Lambda)            (64, 72, 11)              0         
_________________________________________________________________
sequential_3 (Sequential)    (64, 72, 1)               12        
Total params: 12
Trainable params: 12
Non-trainable params: 0
_________________________________________________________________


In [83]:
from keras.layers import*
from keras.models import *
from keras import backend as K
import numpy as np

a = Input(shape=(10, 5), batch_shape=(16,10,5) )
a1 = Input(tensor=K.variable(np.ones((16, 10, 5))), batch_shape=(16,10,5))
x = [a, a1]  # x = [a, a] works fine
b = concatenate(x, 1)
x += [b]  # This changes b._keras_history[0].input
constant=K.variable(np.ones((16,10, 5)))
b = Lambda(lambda x: K.concatenate([x,constant],axis=1),output_shape=(20,5))(a)

model = Model(a, b)
model.summary()
x=model.predict(np.zeros((16,10,5)))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_52 (InputLayer)        (16, 10, 5)               0         
_________________________________________________________________
lambda_7 (Lambda)            (16, 20, 5)               0         
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


In [ ]:
x

In [40]:
def map(f, x, dtype=None, parallel_iterations=10):
    '''
    Apply f to each of the elements in x using the specified number of parallel iterations.

    Important points:
    1. By "elements in x", we mean that we will be applying f to x[0],...x[tf.shape(x)[0]-1].
    2. The output size of f(x[i]) can be arbitrary. However, if the dtype of that output
       is different than the dtype of x, then you need to specify that as an additional argument.
    '''
    if dtype is None:
        dtype = x.dtype

    n = tf.shape(x)[0]
    loop_vars = [
        tf.constant(0, n.dtype),
        tf.TensorArray(dtype, size=n),
    ]
    _, fx = tf.while_loop(
        lambda j, _: j < n,
        lambda j, result: (j + 1, result.write(j, f(x[j]))),
        loop_vars,
        parallel_iterations=parallel_iterations
    )
    return fx.stack()

def jacobian(fx, x, parallel_iterations=10):
    '''
    Given a tensor fx, which is a function of x, vectorize fx (via tf.reshape(fx, [-1])),
    and then compute the jacobian of each entry of fx with respect to x.
    Specifically, if x has shape (m,n,...,p), and fx has L entries (tf.size(fx)=L), then
    the output will be (L,m,n,...,p), where output[i] will be (m,n,...,p), with each entry denoting the
    gradient of output[i] wrt the corresponding element of x.
    '''
    return map(lambda fxi: tf.gradients(fxi, x)[0],
               tf.reshape(fx, [-1]),
               dtype=x.dtype,
               parallel_iterations=parallel_iterations)

In [48]:
from tensorflow.python.ops.parallel_for.gradients import jacobian
J=jacobian(g[0],x)

In [2]:
import tensorflow as tf

gradients = K.gradients(model.output, model.input)    
print(gradients)
#value of gradient for the first x_test
x_test_1 = np.array([x[0]])
sess = K.get_session()
evaluated_gradients_1 = sess.run(gradients[0], feed_dict={modelbd.input: x_test_1})
print(evaluated_gradients_1)

#value of gradient for the second x_test
x_test_2 = np.array([x[1]])
evaluated_gradients_2 = sess.run(gradients[0], feed_dict={model.input: x_test_2})
print(evaluated_gradients_2)

NameError: name 'model' is not defined

In [42]:
#tf.variables_initializer([x0])
x0 = tf.cast(x_test_1, dtype=tf.float32);
#b =;
g = tf.gradients( modelbd(x0), x0)
g0 = sess.run(g)

In [86]:
M=72
def jacobian_tensorflow(x, M=72):    
    jacobian_matrix = []
    for m in range(M):
        # We iterate over the M elements of the output vector
        grad_func = K.gradients(modelbd.get_output_at(4)[:,m], model.input) 
        gradients = sess.run(grad_func, feed_dict={modelbd.get_input_at(0): x})
        jacobian_matrix.append(gradients[0][0,:])
    return np.array(jacobian_matrix)

jacobian_tensorflow(x0)

TypeError: Fetch argument None has invalid type <class 'NoneType'>

In [78]:
#x_test_1[0].reshape(1,x_test_1.size)
x0 = x_test_1[0].reshape(1, x_test_1.shape[1], x_test_1.shape[2])

In [87]:
model = Sequential()                                       # 1d output
model.add(Dense(32, input_shape=(16,)))            # 1d input
model.add(Dense(1))
model.summary()
# Minimize mse
rmsprop = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=1.e-5)
model.compile(loss='mse', optimizer=rmsprop)
model.fit(x, y, batch_size=10, epochs=1000, verbose=0)
#gradients = K.gradients(model.output, model.input)              #Gradient of output wrt the input of the model (Tensor)
#print(gradients)

<tf.Tensor 'bidirectional_1_input:0' shape=(?, 72, 11) dtype=float32>